Check and run Batfish container.

In [1]:
%%bash
if (docker ps -f "ancestor=batfish/batfish" | grep "0.0.0.0:9996-9997" > /dev/null); then
  echo "Batfish container is already running."
else
  docker run -d -p 9997:9997 -p 9996:9996 batfish/batfish
fi

Batfish container is already running.


Run startup.py.

In [2]:
%run startup.py

/home/shino/.pyenv/versions/3.6.5/lib/python3.6/site-packages/pybatfish/client/commands.py:50: UserWarning: Pybatfish public API is being updated, note that API names and parameters will soon change.
  "Pybatfish public API is being updated, note that API names and parameters will soon change.")


Declare flow informations.

In [3]:
src_ip = "192.168.10.1"
dst_ip = "192.168.20.1"
ip_protocol = "TCP"
src_port ="49512"
dst_port = "80"
tcp_flags_ack = TcpFlags(ack=True)
match_ack_bit = MatchTcpFlags(tcp_flags_ack, useAck=True)

outbound_flow = HeaderConstraints(srcIps=src_ip,
                         dstIps=dst_ip,
                         ipProtocols=ip_protocol,
                         srcPorts=src_port,
                         dstPorts=dst_port
                         )
return_flow = HeaderConstraints(srcIps=dst_ip,
                         dstIps=src_ip,
                         ipProtocols=ip_protocol,
                         srcPorts=dst_port,
                         dstPorts=src_port,
                         tcpFlags=match_ack_bit
                         )

Create hosts.

In [4]:
import json

host_file_dir = "networks/example2/hosts/"
src_host_dict = {
                 "hostname": "src_host",
                 "hostInterfaces": {
                   "eth0": {
                     "name": "eth0",
                     "prefix": str(src_ip + "/24"),
                     }
                   }
                 }

dst_host_dict = {
                 "hostname": "dst_host",
                 "hostInterfaces": {
                   "eth0": {
                     "name": "eth0",
                     "prefix": str(dst_ip + "/24"),
                     }
                   }
                 }

f = open("{dir}src_host.json".format(dir=host_file_dir), "w")
f.write(json.dumps(src_host_dict))
f.close()

f = open("{dir}dst_host.json".format(dir=host_file_dir), "w")
f.write(json.dumps(dst_host_dict))
f.close()

Upload configurations.

In [5]:
NETWORK_NAME = "datacenter1"
SNAPSHOT_NAME = "current"
SNAPSHOT_PATH = "networks/example2"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'current'

Validate ACL.

In [6]:
# outbound - expected result
traceroute_ob_ans = bfq.traceroute(
                                headers=outbound_flow,
                                startLocation="src_host",
                                #startLocation="enter(core[GigabitEthernet1/0])"
                               ).answer()
display_html(traceroute_ob_ans.frame())

,Flow,Traces,TraceCount
0,Src IP: 192.168.10.1Src Port: 49512Dst IP: 192.168.20.1Dst Port: 80IP Protocol: TCPStart Location: src_host,"ACCEPTED1. node: src_host ORIGINATED(default) FORWARDED(Routes: static [Network: 0.0.0.0/0, Next Hop IP:AUTO/NONE(-1l)]) TRANSMITTED(eth0)2. node: core RECEIVED(GigabitEthernet1/0: seg10in) FORWARDED(Routes: connected [Network: 192.168.20.0/24, Next Hop IP:AUTO/NONE(-1l)]) TRANSMITTED(GigabitEthernet2/0)3. node: dst_host RECEIVED(eth0) ACCEPTED(InboundStep)",1


In [7]:
# return - unexpected result
traceroute_rt_ans = bfq.traceroute(
                                   headers=return_flow,
                                   startLocation="dst_host",
                                  ).answer()
display_html(traceroute_rt_ans.frame())

BatfishException: Coordinator returned failure: Invalid question datacenter1/traceroute_1dc0e2e6-4713-4455-8b8f-b25a804cc88d: Could not parse JSON question: Cannot deserialize instance of `java.util.HashSet` out of START_OBJECT token
 at [Source: (String)"{"class":"org.batfish.question.traceroute.TracerouteQuestion","differential":false,"headers":{"applications":null,"dscps":null,"dstIps":"192.168.10.1","dstPorts":"49512","ecns":null,"flowStates":null,"fragmentOffsets":null,"icmpCodes":null,"icmpTypes":null,"ipProtocols":["TCP"],"packetLengths":null,"srcIps":"192.168.20.1","srcPorts":"80","tcpFlags":{"tcpFlags":{"ack":true,"cwr":false,"ece":false,"fin":false,"psh":false,"rst":false,"syn":false,"urg":false},"useAck":true,"useCwr":true,"useEce":tru"[truncated 1689 chars]; line: 1, column: 352] (through reference chain: org.batfish.question.traceroute.TracerouteQuestion["headers"]->org.batfish.datamodel.PacketHeaderConstraints["tcpFlags"])